<a href="https://colab.research.google.com/github/Requenamar3/Data-Mining/blob/main/Data_Mining_Module1_Ecommerce_CLV%20filtered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Customer Lifetime Value model

In [1]:
#package used for customer lifetime value modeling
!pip install lifetimes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00


In [2]:
 # installing the pandas_profiling package for data analysis and generating statistical report summaries.
!pip install ydata_profiling


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 23.8 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=090c18b68fb43084f4327493e6ab5d27be9ea9810a2ca5151a1ef3792e9b240b
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.13.1
    Uninstalling seaborn-0.13.1:
      Successfully uninstalled seaborn-0.13.1


In [3]:
# Customer Lifetime Value (CLV or CLTV)
# Buy till you die... (BG/NBD)

# Import the lifetimes package for customer lifetime value analysis.
import lifetimes
# Import pandas for data manipulation and analysis.
import pandas as pd
# Import numpy for numerical computing.
import numpy as np
# Import datetime for handling dates and times.
import datetime as dt
# Import matplotlib.pyplot for creating static, animated, and interactive visualizations.
import matplotlib.pyplot as plt
# Import seaborn for making statistical graphics.
import seaborn as sns
# Import BetaGeoFitter from lifetimes for fitting the BG/NBD model.
from lifetimes import BetaGeoFitter
# Import GammaGammaFitter from lifetimes for fitting the Gamma-Gamma model.
from lifetimes import GammaGammaFitter
# Import plot_frequency_recency_matrix for visualizing the relationship between frequency and recency.
from lifetimes.plotting import plot_frequency_recency_matrix

# Set pandas display options to improve the readability of output.
# Set the maximum number of rows to display to 500.
pd.set_option('display.max_rows', 500)
# Set the maximum number of columns to display to 500.
pd.set_option('display.max_columns', 500)
# Set the width of the display in characters to 1000.
pd.set_option('display.width', 1000)


In [4]:
#Import the ProfileReport .For creating comprehensive exploratory data analysis reports.
from ydata_profiling import ProfileReport

In [5]:
EC= pd.read_csv("https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/sales_2023-10-01_2024-03-06.csv")

In [6]:
EC.head()

,month,financial_status,order_name,customer_email,customer_id,customer_type,variant_sku,product_price,product_title,shipping_region,shipping_postal_code,customer_cohort_week,customer_cohort_quarter,customer_cohort_month,product_type,total_sales,net_sales,total_cost,units_per_transaction
0,2023-10,paid,#805723,mayapriebe33@gmail.com,6616065245299,Returning,SQ9178530,0.00,Bloomsy Original (Ships every 1 Months),Colorado,80525,2023-W19,2023-04,2023-05,NaN,0.00,0.00,0.0,1
1,2023-10,partially_refunded,#810617,vincent.nicolais@yahoo.com,6261751251059,Returning,NaN,0.00,NaN,Georgia,31820,2022-W41,2022-10,2022-10,NaN,83.88,0.00,0.0,0
2,2023-10,partially_refunded,#810617,vincent.nicolais@yahoo.com,6261751251059,Returning,SQ8862610,642.99,Bloomsy Premium,Georgia,31820,2022-W41,2022-10,2022-10,NaN,694.43,642.99,0.0,1
3,2023-10,partially_refunded,#810431,cjsherrillmain@gmail.com,3496267448435,Returning,SQ1726938,69.99,Bloomsy Deluxe,Maryland,20636,2020-W48,2020-10,2020-11,NaN,74.19,69.99,0.0,1
4,2023-10,partially_refunded,#810431,cjsherrillmain@gmail.com,3496267448435,Returning,NaN,0.00,NaN,Maryland,20636,2020-W48,2020-10,2020-11,NaN,-10.60,-10.60,0.0,0


In [7]:
# Select only the desired columns
EC_filtered = ['customer_id', 'order_name','variant_sku','month', 'customer_cohort_month', 'total_sales', 'units_per_transaction']
EC= EC[EC_filtered]
EC.head()


,customer_id,order_name,variant_sku,month,customer_cohort_month,total_sales,units_per_transaction
0,6616065245299,#805723,SQ9178530,2023-10,2023-05,0.00,1
1,6261751251059,#810617,NaN,2023-10,2022-10,83.88,0
2,6261751251059,#810617,SQ8862610,2023-10,2022-10,694.43,1
3,3496267448435,#810431,SQ1726938,2023-10,2020-11,74.19,1
4,3496267448435,#810431,NaN,2023-10,2020-11,-10.60,0


In [8]:
# Create a ProfileReport object
profile = ProfileReport(EC, title="EC CLV", explorative=True)

In [9]:
# display report
profile


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# Remove rows from 'EC' where 'total_sales' is equal to 0.00
EC = EC[EC['total_sales'] != 0.00]


In [11]:
# Remove records with 0 orders qty
EC = EC[EC['units_per_transaction'] != 0]


In [12]:
EC.head()

,customer_id,order_name,variant_sku,month,customer_cohort_month,total_sales,units_per_transaction
2,6261751251059,#810617,SQ8862610,2023-10,2022-10,694.43,1
3,3496267448435,#810431,SQ1726938,2023-10,2020-11,74.19,1
8,5601422671987,#802622,BB738-24,2023-10,2022-03,54.49,1
11,6769237557363,#801731,BB727-26,2023-10,2023-07,64.34,1
13,6272794919027,#807669,BB944-24,2023-10,2022-10,63.74,1


In [13]:
EC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44231 entries, 2 to 158180
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            44231 non-null  int64  
 1   order_name             44231 non-null  object 
 2   variant_sku            44214 non-null  object 
 3   month                  44231 non-null  object 
 4   customer_cohort_month  44231 non-null  object 
 5   total_sales            44231 non-null  float64
 6   units_per_transaction  44231 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 3.7+ MB


In [14]:
EC.isnull().sum()

customer_id               0
order_name                0
variant_sku              17
month                     0
customer_cohort_month     0
total_sales               0
units_per_transaction     0
dtype: int64

In [15]:
EC.dropna(inplace=True)  # inplace=True means we dropped them permanently

In [16]:
EC.isnull().sum()

customer_id              0
order_name               0
variant_sku              0
month                    0
customer_cohort_month    0
total_sales              0
units_per_transaction    0
dtype: int64

In [17]:
EC.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44214 entries, 2 to 158180
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   customer_id            44214 non-null  int64  
 1   order_name             44214 non-null  object 
 2   variant_sku            44214 non-null  object 
 3   month                  44214 non-null  object 
 4   customer_cohort_month  44214 non-null  object 
 5   total_sales            44214 non-null  float64
 6   units_per_transaction  44214 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 2.7+ MB


In [18]:
EC.describe()

,customer_id,total_sales,units_per_transaction
count,4.421400e+04,44214.000000,44214.000000
mean,5.562729e+12,117.220923,1.005406
std,1.682923e+12,149.469024,0.084234
min,1.183560e+12,0.220000,1.000000
25%,5.028425e+12,49.530000,1.000000
50%,6.368478e+12,59.600000,1.000000
75%,6.996533e+12,84.790000,1.000000
max,7.301131e+12,3507.540000,5.000000


***find_boundaries*** function: Calculates upper and lower boundaries (quantiles) for a given variable in the DataFrame.

**capping_outliers function:** Caps outliers in a given variable of the DataFrame by replacing values beyond the upper and lower boundaries with those boundaries.

*These functions are designed to efficiently handle outlier treatment in a DataFrame, making the code concise and easy to understand.*

Outliers are replaced with the upper and lower boundaries calculated based on default quantiles (5th and 95th percentiles).

In [19]:
def find_boundaries(df, variable,q1=0.05,q2=0.95):
    # the boundaries are the quantiles
    lower_boundary = df[variable].quantile(q1) # lower quantile
    upper_boundary = df[variable].quantile(q2) # upper quantile
    return upper_boundary, lower_boundary
def capping_outliers(df,variable):
    upper_boundary,lower_boundary =  find_boundaries(df,variable)
    df[variable] = np.where(df[variable] > upper_boundary, upper_boundary,
                       np.where(df[variable] < lower_boundary, lower_boundary, df[variable]))

In [20]:
# Function to cap outliers in the 'Total_sales'
capping_outliers(EC, 'total_sales')

# Function to cap outliers in the 'units_per_transaction'
capping_outliers(EC, 'units_per_transaction')


In [ ]:
EC.describe()

,customer_id,total_sales,units_per_transaction
count,4.421400e+04,44214.000000,44214.0
mean,5.562729e+12,109.982135,1.0
std,1.682923e+12,118.461722,0.0
min,1.183560e+12,42.790000,1.0
25%,5.028425e+12,49.530000,1.0
50%,6.368478e+12,59.600000,1.0
75%,6.996533e+12,84.790000,1.0
max,7.301131e+12,498.934000,1.0


The average of sales ammount per transaction is around $110. The high standard deviation of $118 indicates the sales amounts vary. The minimum sales is $42.49 and the maximum $498.94.A quater of the transactions are relatively small in total sales. The median indicates that most customers are spending around $59.60 per transaction. This is valuable information for understanding customer behavior, planning inventory and supply chain management, or developing marketing and sales strategies aimed at the most common transaction size.

**Customer Behavior**: Most customers are spending around $59.60 per transaction .This can help us making decisions about product pricing, promotions, and product bundles to cater to the spending habits of the majority.

**Marketing Strategies**: Marketing efforts can be tailored to the average customer's spending. If most customers spend around $59.60, marketing can highlight products or deals around this price. Upselling strategies could focus on slightly higher-priced items to increase the average spend.
We could recommend additional purchases of accesories, with the goal of slightly increasing the average spend per customer.

#Recency Frecuency Time

In [ ]:
# Calculate and assign the floor division of 'total_sales' by 'ordered_item_quantity' to 'UnitPrice' column in EC DataFrame.
EC['UnitPrice'] = EC['total_sales'] // EC['units_per_transaction']


In [ ]:
EC.columns

Index(['customer_id', 'order_name', 'variant_sku', 'month', 'customer_cohort_month', 'total_sales', 'units_per_transaction', 'UnitPrice'], dtype='object')

In [ ]:
# Compute summary customer lifetime value (CLV) data from transaction data
clv = lifetimes.utils.summary_data_from_transaction_data(EC,'customer_id','month','total_sales',observation_period_end='2024-03-05')

In [ ]:
clv.head()

,frequency,recency,T,monetary_value
customer_id,,,,
1183559942259,2.0,62.0,95.0,85.59
1183569084531,1.0,30.0,125.0,42.79
1183569707123,5.0,152.0,156.0,47.07
1183569805427,0.0,0.0,64.0,0.00
1183579504755,0.0,0.0,64.0,0.00


In [ ]:
# let's filter so that we only catch customers who have made more than 1 purchase
clv = clv[clv['frequency']>1]

In [ ]:
clv.head()

,frequency,recency,T,monetary_value
customer_id,,,,
1183559942259,2.0,62.0,95.0,85.59
1183569707123,5.0,152.0,156.0,47.07
1183582322803,4.0,123.0,156.0,53.49
1183584551027,5.0,152.0,156.0,43.79
1183594184819,5.0,152.0,156.0,43.99


#BG/NBD Model

Initializes and fits the Beta-Geometric/Beta-Binomial (BG/NBD) model to the provided summary statistics. These statistics include the frequency (number of repeat transactions), recency (age at the last transaction), and tenure (age at the end of the observation period) for each customer. The BG/NBD model is a probabilistic model used for predicting customer lifetime value (CLV) and future purchase behavior based on historical transaction data. The penalizer coefficient is specified to prevent overfitting during model fitting.

In [ ]:
# Initialize the Beta-Geometric/Beta-Binomial model (BG/NBD) with a penalizer coefficient to avoid overfitting
bgf = BetaGeoFitter(penalizer_coef=0.001)

# Fit the BG/NBD model using the summary statistics: frequency (F), recency (R), and tenure (T)
bgf.fit(
    clv['frequency'],  # Frequency represents the number of repeat transactions
    clv['recency'],    # Recency represents the age of the customer at their last transaction
    clv['T']           # Tenure represents the age of the customer at the end of the observation period
)


/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 4404 subjects, a: 0.00, alpha: 322.63, b: 0.00, r: 8.02>

Predicts the expected number of purchases for each customer in the next 6 months using the Beta-Geometric/Beta-Binomial (BG/NBD) model. It calculates these predictions based on the customer's historical repeat transaction behavior, considering their frequency (number of repeat transactions), recency (age at their last transaction), and tenure (age at the end of the observation period). Then, it sorts the customers by their expected purchases in descending order and selects the top 5 customers with the highest expected purchases for further analysis or action.

In [ ]:
t = 180  # Define the prediction period of 6 months (180 days)

# Predict future purchases for the next 6 months using the BG/NBD model
clv['expected_purc_6_months'] = bgf.conditional_expected_number_of_purchases_up_to_time(
    t,                      # Prediction period
    clv['frequency'],       # Frequency represents the number of repeat transactions
    clv['recency'],         # Recency represents the age of the customer at their last transaction
    clv['T']                # Tenure represents the age of the customer at the end of the observation period
)

# Sort customers by expected purchases in the next 6 months and display the top 5
clv_top5 = clv.sort_values(by='expected_purc_6_months', ascending=False).head(5)


In [ ]:
clv_top5

,frequency,recency,T,monetary_value,expected_purc_6_months
customer_id,,,,,
5454865727603,5.0,152.0,156.0,112.384,4.896924
5156199071859,5.0,152.0,156.0,87.088,4.896924
5202694045811,5.0,152.0,156.0,49.150,4.896924
5200096985203,5.0,152.0,156.0,62.988,4.896924
5188042293363,5.0,152.0,156.0,53.490,4.896924


#Gamma-Gamma Model

In [ ]:
# Calculate and display the correlation between 'frequency' and 'monetary_value' columns in the 'clv' DataFrame.
clv[['frequency', 'monetary_value']].corr()


,frequency,monetary_value
frequency,1.000000,-0.169663
monetary_value,-0.169663,1.000000


In [ ]:
# Initialize GammaGammaFitter with a penalizer to prevent overfitting and fit on frequency and monetary value.
ggf = GammaGammaFitter(penalizer_coef=0.01)  # Create GammaGammaFitter model instance with a specified penalizer.
ggf.fit(clv["frequency"], clv["monetary_value"])  # Fit model to customer data for predicting monetary value.


<lifetimes.GammaGammaFitter: fitted with 4404 subjects, p: 4.36, q: 0.66, v: 4.26>

In [ ]:
# Calculate 6-month Customer Lifetime Value (CLV) using the GammaGamma and BetaGeo models, then identify top customers.
# The 'customer_lifetime_value' method from GammaGammaFitter ('ggf') combines both frequency/recency (from 'bgf') and
# monetary value to estimate the CLV for the next 6 months ('time=6'), assuming daily ('freq='D'') transactions.
# A discount rate of 0.01 is applied to account for the time value of money.
clv['six_Months_CLV']=ggf.customer_lifetime_value(bgf,
                                   clv["frequency"],
                                   clv["recency"],
                                   clv["T"],
                                   clv["monetary_value"],
                                   time=6,
                                   freq='D',
                                   discount_rate=0.01)
clv.sort_values('six_Months_CLV',ascending=False).head()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV,Segment
customer_id,,,,,,,
6543258091635,5.0,152.0,156.0,1194.816,4.896924,5744.056762,Champions
6569497985139,2.0,123.0,156.0,1227.343,3.768707,4654.972431,Champions
5419749965939,2.0,91.0,95.0,997.868,4.319171,4339.157212,Champions
5455684370547,2.0,92.0,156.0,803.544,3.768707,3050.406054,Champions
5394003755123,2.0,62.0,95.0,606.174,4.319171,2639.533688,Champions


In [ ]:
# Segment customers into 4 categories based on their CLV using quartiles.
clv['Segment'] = pd.qcut(clv['six_Months_CLV'], 4, labels=['Hibernating', 'Need Attention', 'Loyal Customers', 'Champions'])  # Assign segments based on CLV quartiles.


In [ ]:
clv.head()

,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV,Segment
customer_id,,,,,,,
1183559942259,2.0,62.0,95.0,85.59,4.319171,380.635638,Champions
1183569707123,5.0,152.0,156.0,47.07,4.896924,230.221868,Need Attention
1183582322803,4.0,123.0,156.0,53.49,4.520851,242.910696,Need Attention
1183584551027,5.0,152.0,156.0,43.79,4.896924,214.464568,Hibernating
1183594184819,5.0,152.0,156.0,43.99,4.896924,215.425378,Hibernating


In [ ]:
# Calculate the mean of each column within the 'clv' DataFrame grouped by 'Segment'.
clv.groupby('Segment').mean()


,frequency,recency,T,monetary_value,expected_purc_6_months,six_Months_CLV
Segment,,,,,,
Hibernating,3.750226,117.220814,155.078733,47.117612,4.435272,211.100414
Need Attention,3.985441,125.137398,154.201092,54.114294,4.531328,245.963066
Loyal Customers,3.351228,106.268426,136.388535,66.896035,4.454906,300.817684
Champions,3.584015,117.536785,145.468665,146.735710,4.460182,645.252504


#Check out top customers

In [ ]:
# Filter out the unwanted product titles before the groupby operation
EC_filtered = EC[~EC['variant_sku'].isin(['SQ3100795', 'SQ5173256'])]

# Group by 'customer_id', 'customer_cohort_month', and 'variant_sku' then count the number of records
customer_record_counts = EC_filtered.groupby(['customer_id', 'customer_cohort_month', 'variant_sku']).size().reset_index(name='record_count')

# Get the top 15 customer IDs by record count in descending order
top_customers = customer_record_counts.nlargest(15, 'record_count')

# drop duplicates
top_customers = top_customers.drop_duplicates(subset='customer_id')

# Merge top_customers with the filtered dataframe to include customer emails and product titles.
# left join to ensure all top customers are included in the final DataFrame even if they have no matching records in EC_filtered
top_customers_info = top_customers.merge(
    EC_filtered[['customer_id', 'customer_cohort_month', 'variant_sku', 'customer_email', 'product_title']],
    on=['customer_id', 'customer_cohort_month', 'variant_sku'],
    how='left'
)

# Select columns we're interested in and drop duplicates
top_customers_info = top_customers_info[['customer_id', 'customer_email', 'product_title', 'variant_sku', 'customer_cohort_month', 'record_count']].drop_duplicates(subset='customer_id')

print(top_customers_info)



KeyError: "['customer_email', 'product_title'] not in index"

In [ ]:
EC.describe()

,customer_id,total_sales,units_per_transaction,UnitPrice
count,4.421400e+04,44214.000000,44214.0,44214.000000
mean,5.562729e+12,109.982135,1.0,109.400755
std,1.682923e+12,118.461722,0.0,118.416554
min,1.183560e+12,42.790000,1.0,42.000000
25%,5.028425e+12,49.530000,1.0,49.000000
50%,6.368478e+12,59.600000,1.0,59.000000
75%,6.996533e+12,84.790000,1.0,84.000000
max,7.301131e+12,498.934000,1.0,498.000000


In [ ]:
EC[]

SyntaxError: invalid syntax (<ipython-input-45-c53276ff8bdb>, line 1)

#Add table from Ship Station in order to look up fo the missing information

In [ ]:
# Load the dataset from the provided URL
ss_df = pd.read_csv('https://raw.githubusercontent.com/Requenamar3/Data-Mining/main/SS.csv')

# Display the first few rows of the dataframe to understand its structure
ss_df.head()

,Date - Shipped Date,Market - Store Name,Market - Markeplace Name,Date - Order Date,Custom - Field 3,Customer Email,Ship To - City,Custom - Field 1,Custom - Field 2,Item - SKU,Ship To - Postal Code,Order - Number,Item - Name,Ship To - State,Item - Qty,Carrier - Service Selected
0,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,8/3/2023 2:49:14 PM,11-01-2023,JDJVJ@AOL.COM,FT LAUDERDALE,4.MIAMI,ORIGINAL_PLEGABLE,SQ9178530,33316-3228,776024,ORIGINAL,FL,1,FedEx Home Delivery®
1,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,8/22/2023 9:48:07 AM,11-01-2023,mary.c.parker@ttuhsc.edu,LUBBOCK,2.TEXAS,DELUXE,BB-SG-MIXED,79416-1340,783802,DELUXE,TX,1,FedEx Home Delivery®
2,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,9/4/2023 12:56:49 PM,11-01-2023,ajbroc@me.com,CHAPPAQUA,3.NEW JERSEY,DELUXE,BB-SG-MIXED,10514-1302,789981,DELUXE,NY,1,FedEx Home Delivery®
3,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,9/5/2023 11:59:05 AM,11-01-2023,bndudley52@gmail.com,SCOTTSDALE,5.CALIFORNIA,PREMIUM,SQ8862610,85255-7165,790118,PREMIUM,AZ,1,OnTrac Ground Service
4,10/31/2023 12:00:00 AM,BloomsyBox Shopify,Shopify,9/5/2023 11:59:05 AM,11-01-2023,bndudley52@gmail.com,SCOTTSDALE,5.CALIFORNIA,PREMIUM,NaN,85255-7165,790118,cs15,AZ,1,OnTrac Ground Service


In [ ]:
ss_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469 entries, 0 to 468
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Date - Shipped Date         469 non-null    object
 1   Market - Store Name         469 non-null    object
 2   Market - Markeplace Name    469 non-null    object
 3   Date - Order Date           469 non-null    object
 4   Custom - Field 3            469 non-null    object
 5   Customer Email              446 non-null    object
 6   Ship To - City              469 non-null    object
 7   Custom - Field 1            451 non-null    object
 8   Custom - Field 2            377 non-null    object
 9   Item - SKU                  455 non-null    object
 10  Ship To - Postal Code       469 non-null    object
 11  Order - Number              469 non-null    object
 12  Item - Name                 469 non-null    object
 13  Ship To - State             469 non-null    object

In [ ]:
ss_df.columns


Index(['Date - Shipped Date', 'Market - Store Name', 'Market - Markeplace Name', 'Date - Order Date', 'Custom - Field 3', 'Customer Email', 'Ship To - City', 'Custom - Field 1', 'Custom - Field 2', 'Item - SKU', 'Ship To - Postal Code', 'Order - Number', 'Item - Name', 'Ship To - State', 'Item - Qty', 'Carrier - Service Selected'], dtype='object')

In [ ]:
# Merge EC_df with ss_df on the 'order_name' column in EC_df and 'Order Number' in ss_df
# Make sure the column names are exactly as they appear in your dataframes
ECSS = EC.merge(ss_df[['Custom - Field 3', 'Customer Email', 'Item - SKU', 'Order - Number', 'Item - Name', 'Ship To - State', 'Carrier - Service Selected']],
                        left_on='order_name', right_on='Order - Number', how='left')

# # Now merged_df has the original EC_df data along with the additional columns from ss_df
# # You can then drop the 'Order - Number' column if it's no longer needed
# merged_df.drop('Order - Number', axis=1, inplace=True)

# # If you only want to keep rows where data was successfully merged and no NaN values exist in the new columns, you can do this:
# merged_df.dropna(subset=['Custom - Field 3', 'Customer Email', 'Item - SKU', 'Item - Name', 'Ship To - State', 'Carrier - Service Selected'], inplace=True)

# Save the merged dataframe to a new CSV file or use it for further analysis
# merged_df.to_csv('merged_dataframe.csv', index=False)

ECSS.head()  # Display the first few rows of the merged dataframe


,month,financial_status,order_name,customer_email,customer_type,customer_id,variant_sku,product_title,shipping_postal_code,shipping_region,customer_cohort_month,total_sales,ordered_item_quantity,Custom - Field 3,Customer Email,Item - SKU,Order - Number,Item - Name,Ship To - State,Carrier - Service Selected
0,2023-11,paid,826267,amansndrs@gmail.com,Returning,6816167329907,NaN,NaN,71254,Louisiana,2023-08,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-11,paid,821120,jsajcat@gmail.com,Returning,6882693939315,SQ1726938,Bloomsy Deluxe,60101,Illinois,2023-09,76.98,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-11,paid,825181,amymattey@gmail.com,Returning,3492230627443,NaN,NaN,44035,Ohio,2020-11,83.88,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-11,partially_refunded,793583,kathryndonnelly@yahoo.com,Returning,6536856797299,NaN,NaN,60555,Illinois,2023-03,-162.24,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-11,partially_refunded,814967,nancynetherland@gmail.com,Returning,5686113435763,NaN,NaN,98502,Washington,2022-05,-91.44,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
ECSS.columns

Index(['month', 'financial_status', 'order_name', 'customer_email', 'customer_type', 'customer_id', 'variant_sku', 'product_title', 'shipping_postal_code', 'shipping_region', 'customer_cohort_month', 'total_sales', 'ordered_item_quantity', 'Custom - Field 3', 'Customer Email', 'Item - SKU', 'Order - Number', 'Item - Name', 'Ship To - State', 'Carrier - Service Selected'], dtype='object')

In [ ]:
ECSS.isnull().sum()

month                              0
financial_status                   0
order_name                         0
customer_email                  3049
customer_type                      0
customer_id                        0
variant_sku                    63281
product_title                  63264
shipping_postal_code               1
shipping_region                    1
customer_cohort_month              0
total_sales                        0
ordered_item_quantity              0
Custom - Field 3              131970
Customer Email                131970
Item - SKU                    131970
Order - Number                131970
Item - Name                   131970
Ship To - State               131970
Carrier - Service Selected    131970
dtype: int64

In [ ]:
df.dropna(inplace=True)  # inplace=True means we dropped them permanently

In [ ]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64